In [1]:
from datasets import load_dataset
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor,AutoProcessor, AutoModelForCausalLM
import torch
from evaluate import load

torch.cuda.empty_cache()

dataset = load_dataset('eduvedras/Img_Vars',split='test',trust_remote_code=True)


/home/eduvedras/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

filetag = "pix2struct-textcaps-base-desc-vars-final"

model = Pix2StructForConditionalGeneration.from_pretrained(f"eduvedras/{filetag}").to(device)
#model = AutoModelForCausalLM.from_pretrained(f"eduvedras/{filetag}").to(device)
model

Pix2StructForConditionalGeneration(
  (encoder): Pix2StructVisionModel(
    (embeddings): Pix2StructVisionEmbeddings(
      (patch_projection): Linear(in_features=768, out_features=768, bias=True)
      (row_embedder): Embedding(4096, 768)
      (column_embedder): Embedding(4096, 768)
      (dropout): Dropout(p=0.06, inplace=False)
    )
    (encoder): Pix2StructVisionEncoder(
      (layer): ModuleList(
        (0-11): 12 x Pix2StructVisionLayer(
          (attention): Pix2StructVisionAttention(
            (query): Linear(in_features=768, out_features=768, bias=False)
            (key): Linear(in_features=768, out_features=768, bias=False)
            (value): Linear(in_features=768, out_features=768, bias=False)
            (output): Linear(in_features=768, out_features=768, bias=False)
          )
          (mlp): Pix2StructVisionMlp(
            (wi_0): Linear(in_features=768, out_features=2048, bias=False)
            (wi_1): Linear(in_features=768, out_features=2048, bias=False)


In [3]:
checkpoint = "google/pix2struct-textcaps-base"
#checkpoint = "microsoft/git-base"

processor = Pix2StructProcessor.from_pretrained(checkpoint)
#processor = AutoProcessor.from_pretrained(checkpoint)

predictions = []
references = []

dic = {}

from tqdm.auto import tqdm

for i in range(len(dataset)):
    dic[dataset[i]['Chart_name']] = []

for i in range(len(dataset)):
    dic[dataset[i]['Chart_name']].append(dataset[i]['Description'])
    
print(len(dic))

15


In [4]:
for i in tqdm(range(len(dataset))):
    inputs = processor(images=dataset[i]['Chart'], return_tensors="pt").to(device)
    flattened_patches = inputs.flattened_patches
    #pixel_values = inputs.pixel_values
    attention_mask = inputs.attention_mask
    generated_ids = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=167)
    
    #generated_ids = model.generate(pixel_values=pixel_values, max_length=167)
    
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]  
    predictions.append(generated_caption)
    references.append(dic[dataset[i]['Chart_name']])

100%|██████████| 15/15 [00:08<00:00,  1.68it/s]


In [5]:
import evaluate


file = open(f"{filetag}.txt", "a")

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)
file.write(f"BLEU: {results}\n")

meteor = evaluate.load("meteor")
results = meteor.compute(predictions=predictions, references=references)
print(results)
file.write(f"METEOR: {results}\n")

rouge = evaluate.load("rouge")
results = rouge.compute(predictions=predictions, references=references)
print(results)
file.write(f"ROUGE: {results}\n")
file.close()

{'bleu': 0.0, 'precisions': [0.10975609756097561, 0.0, 0.0, 0.0], 'brevity_penalty': 0.05227591594298973, 'length_ratio': 0.25308641975308643, 'translation_length': 82, 'reference_length': 324}


[nltk_data] Downloading package wordnet to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/eduvedras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/eduvedras/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'meteor': 0.012753232902160168}
{'rouge1': 0.051870525988173044, 'rouge2': 0.007141179234202489, 'rougeL': 0.03997510115157174, 'rougeLsum': 0.0406411453470277}


In [6]:
import pandas as pd

new_df = pd.DataFrame(columns=['Image','Prediction'])

i=0
while i < len(dataset):
    if ((new_df['Image'] == dataset[i]['Chart_name']) & (new_df['Prediction'] == predictions[i])).any():
        i += 1
        continue
    else:
        new_df.loc[len(new_df)] = {'Image': dataset[i]['Chart_name'], 'Prediction': predictions[i]}
        i += 1
    
new_df.to_csv(f'{filetag}.csv', index=False)